In [10]:
import pandas as pd

# Load data
workers = pd.read_csv('workers.csv')
job_locations = pd.read_csv('job_locations.csv')

# Example preprocessing steps
# Convert categorical variables to numerical ones (e.g., skill set, language proficiency)
workers['skill_set'] = workers['skill_set'].astype('category').cat.codes
workers['language_proficiency'] = workers['language_proficiency'].astype('category').cat.codes
job_locations['required_skill_set'] = job_locations['required_skill_set'].astype('category').cat.codes
# # Calculate proximity (assuming lat/lon data for simplicity)
# def calculate_distance(lat1, lon1, lat2, lon2):
#     from geopy.distance import geodesic
#     return geodesic((lat1, lon1), (lat2, lon2)).km

# job_locations['proximity'] = job_locations.apply(
#     lambda row: calculate_distance(row['lat'], row['lon'], workers['lat'], workers['lon']), axis=1
# )


In [11]:
# Create features for matching
features = pd.DataFrame()
features['worker_id'] = workers['worker_id']
features['job_location_id'] = job_locations['job_location_id']
features['skill_match'] = (workers['skill_set'] == job_locations['required_skill_set']).astype(int)
# features['proximity'] = job_locations['proximity']
features['cost'] = workers['wage_rate']
print(features)

# You can add more features as necessary


   worker_id  job_location_id  skill_match  cost
0          1              101         True    25
1          2              102         True    30
2          3              103         True    28
3          4              104         True    35
4          5              105         True    26


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Assume you have a label indicating successful matches
# features['label'] = ... # Your label data here

# Split data
X = features.drop('skill_match', axis=1)
y = features['skill_match']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))


Accuracy: 1.0


In [6]:
def calculate_productivity(experience_level, past_performance, health_status):
    # Assign numeric values to experience levels and health statuses
    experience_value = {'Beginner': 0.8, 'Intermediate': 1.0, 'Advanced': 1.2}
    health_value = {'Healthy': 1.0, 'Minor Issues': 0.8, 'Major Issues': 0.5}
    
    # Calculate productivity score
    score = (experience_value[experience_level] * past_performance * health_value[health_status])
    return round(score, 2)

Generate workers data (generated_workers.csv)

In [51]:
import pandas as pd
from faker import Faker
import random

fake = Faker()

# Define the number of records
num_records = 2000

# Predefined lists for random choices
skill_sets = ['Plumbing', 'Electrical', 'Carpentry', 'Painting', 'Landscaping', 'HVAC', 'Roofing', 'Drywall', 'Flooring', 'Heavy Machinery Operation']
experience_levels = ['Beginner', 'Intermediate', 'Advanced']
health_statuses = ['Healthy', 'Minor Issues', 'Major Issues']
job_sites = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
# training_statuses = ['Completed', 'In Progress', 'Not Started']
# work_conditions = ['Indoor', 'Outdoor', 'High-altitude', 'Underwater']

def generate_record(worker_id):
    experience_level = random.choice(experience_levels)
    past_performance = random.randint(1, 10)  # Assuming 1 to 10 scale
    health_status = random.choice(health_statuses)
    
    return {
        'worker_id': worker_id,
        'name': fake.name(),
        'skill_set': random.choice(skill_sets),
        'experience_level': experience_level,
        'availability': random.choice(['Available', 'Unavailable']),
        'health_status': health_status,
        'past_performance': past_performance,
        # 'wage_rate': round(random.uniform(15, 50), 2),  # Assuming hourly rate in dollars
        # 'job_sites': random.choice(job_sites),  # New column for job sites
        'productivity_score': calculate_productivity(experience_level, past_performance, health_status)  # New column for productivity score
    }

# Generate records
records = [generate_record(i+1) for i in range(num_records)]

# Convert to DataFrame
df = pd.DataFrame(records)

# Save to CSV
df.to_csv('generated_workers.csv', index=False)

In [17]:
import random
import csv

# Assuming skill_set is a list of skills you want to use as keys
skill_set = ['Plumbing', 'Electrical', 'Carpentry', 'Painting', 'Landscaping', 'HVAC', 'Roofing', 'Drywall', 'Flooring', 'Heavy Machinery Operation']

# Generating a dictionary with skills as keys and random days (1-100) as values
skill_days_dict = {skill: random.randint(1, 100) for skill in skill_set}

job_sites = [chr(i) for i in range(ord('A'), ord('J')+1)]  # Generates list ['A', 'B', ..., 'J']
safety_requirements = ['low', 'med', 'high']

import statistics

records = []
for job_site in job_sites:
    unique_skill_days_dict = {skill: random.randint(1, 100) for skill in skill_set}
    skills_list = [{'skill': skill, 'days': days} for skill, days in unique_skill_days_dict.items()]
    
    mean_days = statistics.mean([skill['days'] for skill in skills_list])
    # Calculate the percentage of work done
    percentage_work_done = 100 - mean_days
    
    record = {
        'job_site': job_site,
        'skill_days_dict': skills_list,  # This will now be a unique list of dictionaries for each job site
        'safety_requirements': random.choice(safety_requirements),
        'percentage_work_done': percentage_work_done  # New field showing the percentage of work done
    }
    records.append(record)

with open('job_site.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    # Update the header row to include the new column
    writer.writerow(['job_site', 'skill_days_dict', 'safety_requirements', 'percentage_work_done'])
    for record in records:
        # Converting the list of dictionaries to a string for CSV writing
        skill_days_str = '; '.join([f"{skill['skill']}: {skill['days']}" for skill in record['skill_days_dict']])
        # Write the new field to the CSV
        writer.writerow([record['job_site'], skill_days_str, record['safety_requirements'], f"{record['percentage_work_done']:.2f}"])

In [42]:
import pandas as pd
import os

# Create a directory to store the ranks folder
os.makedirs('ranks', exist_ok=True)

# Read the generated workers data from the CSV file
df = pd.read_csv('generated_workers.csv')

# Group the workers by skill set
grouped = df.groupby('skill_set')

for skill_set, group in grouped:
    # Filter the group for available workers before ranking
    available_group = group[group['availability'] == 'Available']
    
    # Sort the available group by productivity score in descending order
    ranked_group = available_group.sort_values('productivity_score', ascending=False)
    
    # Create a separate folder for each skill set
    folder_path = os.path.join('ranks', skill_set)
    os.makedirs(folder_path, exist_ok=True)
    
    # Save the ranked group to a CSV file in the corresponding folder
    file_path = os.path.join(folder_path, 'skill_rank.csv')
    ranked_group.to_csv(file_path, index=False)

In [34]:
import pandas as pd

df_job_sites = pd.read_csv('job_site.csv')
ranked_job_sites = df_job_sites.sort_values('percentage_work_done', ascending=True)

# Initialize a list to store job site ID, skill, and percentage
skill_percentages = []

# Iterate through each row in the DataFrame
for index, row in ranked_job_sites.iterrows():
    # Split the string by ';' to get individual skill-day pairs
    skill_days_pairs = row['skill_days_dict'].split(';')
    
    # Initialize a variable to store total days for the current job site
    total_days_current_job_site = sum(int(pair.split(':')[1].strip()) for pair in skill_days_pairs)
    
    # Iterate through the pairs again to calculate the percentage for each skill
    for pair in skill_days_pairs:
        skill, days = pair.split(':')
        skill = skill.strip()
        days = int(days.strip())
        
        # Calculate the percentage of days for the current skill
        skill_percentage = (days / total_days_current_job_site) * 100 if total_days_current_job_site > 0 else 0
        
        # Append the job site ID (or name), skill, and percentage to the list
        skill_percentages.append([row['job_site'], skill, skill_percentage])  # Assuming there's a 'job_site_id' column

# Convert the list to a DataFrame
df_skill_percentages = pd.DataFrame(skill_percentages, columns=['Job Site ID', 'skill', 'Percentage'])

print(df_skill_percentages)

   Job Site ID                      skill  Percentage
0            I                   Plumbing    6.630582
1            I                 Electrical   12.043302
2            I                  Carpentry    7.713126
3            I                   Painting   12.178620
4            I                Landscaping   13.125846
..         ...                        ...         ...
95           A                       HVAC   15.789474
96           A                    Roofing    0.250627
97           A                    Drywall   22.807018
98           A                   Flooring   23.057644
99           A  Heavy Machinery Operation    0.751880

[100 rows x 3 columns]


In [38]:
# Group by 'Skill' and then apply sorting within each group by 'Percentage'
df_skills_sorted = df_skill_percentages.groupby('skill').apply(lambda x: x.sort_values('Percentage', ascending=False))

# Reset index to clean up the DataFrame
df_skills_sorted = df_skills_sorted.reset_index(drop=True)

print(df_skills_sorted)

   Job Site ID      skill  Percentage
0            J  Carpentry   15.473888
1            A  Carpentry   14.786967
2            B  Carpentry   13.718412
3            G  Carpentry   11.669659
4            E  Carpentry   10.924370
..         ...        ...         ...
95           I    Roofing   10.825440
96           C    Roofing    7.425743
97           H    Roofing    6.688963
98           D    Roofing    4.262877
99           A    Roofing    0.250627

[100 rows x 3 columns]


/tmp/ipykernel_764/3720486569.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_skills_sorted = df_skill_percentages.groupby('skill').apply(lambda x: x.sort_values('Percentage', ascending=False))


In [58]:
import pandas as pd

# Assuming df_skills_sorted is already defined and loaded

# Initialize a dictionary to hold the worker allocations
print(df_skills_sorted.columns)
worker_allocations = {}

# Iterate through each skill in the sorted DataFrame
for index, row in df_skills_sorted.iterrows():
    skill_req = row['skill']
    percentage = row['Percentage']
    site_id = row['Job Site ID']
    skill_rank_df = pd.read_csv(f'ranks/{skill_req}/skill_rank.csv')

    num_workers_needed = int((percentage / 100) * len(skill_rank_df))

    allocated_workers = 0
    for _, worker_row in skill_rank_df.iterrows():
        worker_id = worker_row['worker_id']  # Adjust based on the actual column name in skill_rank_df

        # Check if the worker is already allocated (in any site)
        if worker_id not in [worker for site_workers in worker_allocations.values() for worker in site_workers]:
            # Allocate the worker to the current site
            if site_id not in worker_allocations:
                worker_allocations[site_id] = []
            worker_allocations[site_id].append(worker_id)
            allocated_workers += 1

            # Break the loop if the required number of workers has been allocated
            if allocated_workers >= num_workers_needed:
                break

# print(worker_allocations)
worker_to_job_site = {worker_id: site_id for site_id, workers in worker_allocations.items() for worker_id in workers}

print(worker_to_job_site)

Index(['Job Site ID', 'skill', 'Percentage'], dtype='object')
{762: 'J', 1733: 'J', 983: 'J', 851: 'J', 501: 'J', 382: 'J', 1955: 'J', 958: 'J', 1214: 'J', 1375: 'J', 919: 'J', 859: 'J', 867: 'J', 801: 'J', 254: 'J', 1829: 'J', 1000: 'J', 1302: 'J', 256: 'J', 999: 'J', 1972: 'J', 834: 'J', 1400: 'J', 265: 'J', 1034: 'J', 1571: 'J', 1624: 'J', 1456: 'J', 565: 'J', 165: 'J', 1758: 'J', 1708: 'J', 362: 'J', 700: 'J', 267: 'J', 368: 'J', 1336: 'J', 1553: 'J', 892: 'J', 1985: 'J', 916: 'J', 1788: 'J', 386: 'J', 824: 'J', 1472: 'J', 713: 'J', 1334: 'J', 1500: 'J', 1976: 'J', 1907: 'J', 839: 'J', 1346: 'J', 1593: 'J', 1716: 'J', 261: 'J', 1496: 'J', 1255: 'J', 1055: 'J', 490: 'J', 560: 'J', 1536: 'J', 1740: 'J', 404: 'J', 566: 'J', 464: 'J', 571: 'J', 106: 'J', 955: 'J', 183: 'J', 1530: 'J', 1015: 'J', 1157: 'J', 191: 'J', 271: 'J', 909: 'J', 630: 'J', 1121: 'J', 1455: 'J', 587: 'J', 1878: 'J', 1897: 'J', 428: 'J', 300: 'J', 46: 'J', 1951: 'J', 456: 'J', 844: 'J', 409: 'A', 335: 'A', 1614: 'A

In [59]:
import pandas as pd

# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(worker_to_job_site.items()), columns=['worker_id', 'job_site'])

# Export the DataFrame to a CSV file
df.to_csv('map.csv', index=False)

In [63]:
import pandas as pd

df = pd.read_csv('generated_workers.csv')

# Filter rows with availability as "Available"
df_filtered = df[df['availability'] == 'Available']
df_filtered['job_site'] = df_filtered['worker_id'].map(worker_to_job_site).fillna('NULL')

# Export the filtered and mapped data to train_data.csv
df_filtered.to_csv('train_data.csv', index=False)

/tmp/ipykernel_764/4072989052.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['job_site'] = df_filtered['worker_id'].map(worker_to_job_site).fillna('NULL')


In [65]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('train_data.csv')

# Count the non-NULL values in the "job_sites" column
total_req_members = df['job_site'].notnull().sum()

print("Total required members:", total_req_members)

Total required members: 421
